In [ ]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
chromadb.__version__

In [ ]:
def create_chunks(body, chunk_size, overlap, title="", end_tokens=["\n", " "]):
    count = 0
    word_end_queue = []
    chunks = []
    last_i = 0
    for i in range(len(body)):
        for token in end_tokens:
            if body[i] == token:
                count += 1
                word_end_queue.append(i)
                if len(word_end_queue) >= overlap + chunk_size:
                    word_end_queue.pop(0)
                break
        if count == chunk_size:
            last_i = word_end_queue[len(word_end_queue)-1 - overlap]
            if title == "":
                chunks.append(body[word_end_queue[0]:i])
            else:
                chunks.append("From \" {0}\": ".format(title) + body[word_end_queue[0]:i])
            count = overlap
    if title == "":
        chunks.append(body[last_i:len(body)])
    else:
        chunks.append("From \" {0}\": ".format(title) + body[last_i:len(body)])
    return chunks

In [ ]:
chunk_size = 400
overlap = 10
data_folder = "../data_txt/"
database_folder = "../historical_embeddings"

In [ ]:
studies = {}
files = os.listdir(data_folder)
for file_name in files:
    if file_name[-4:] != ".txt":
        continue
    with open(data_folder + file_name, 'r') as file:
        text = file.read()
    studies[file_name] = create_chunks(text, chunk_size, overlap, title=file_name)

In [ ]:
class EmbeddingClass(EmbeddingFunction):
    def __init__(self):
        self.model = SentenceTransformer("intfloat/multilingual-e5-base")
    def __call__(self, texts: list[str]) -> list[list[float]]:
        return self.model.encode(texts).tolist()

In [ ]:
client = chromadb.PersistentClient(path=database_folder)
collection = client.get_or_create_collection("historical_docs", embedding_function=EmbeddingClass())
last_unused_id = 0
for name in studies.keys():
    documents = studies[name]
    ids = [str(x) for x in range(last_unused_id, len(documents) + last_unused_id)]
    last_unused_id = int(ids[len(ids)-1])+1
    meta_datas = [{"source" : name} for x in range(len(documents))]
    collection.add(ids=ids, documents=documents, metadatas=meta_datas)


In [ ]:
# Loading Table 
import os
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
chromadb.__version__

class EmbeddingClass(EmbeddingFunction):
    def __init__(self):
        self.model = SentenceTransformer("intfloat/multilingual-e5-base")
    def __call__(self, texts: list[str]) -> list[list[float]]:
        return self.model.encode(texts).tolist()

database_folder = "../historical_embeddings"

client = chromadb.PersistentClient(path=database_folder)
collection = client.get_or_create_collection("historical_docs", embedding_function=EmbeddingClass())